# Analysis: Does each BasicActivity belong to exactly one Category only?

In [13]:
;with BasicActivities_with_more_than_one_Category as (
    select BasicActivity 
    from UserActivity 
    group by BasicActivity 
    having count(distinct Category) > 1
)
select ua.Category, ua.BasicActivity, count(*)
from UserActivity ua
inner join BasicActivities_with_more_than_one_Category t
    ON ua.BasicActivity = t.BasicActivity
group by ua.Category, ua.BasicActivity

(2 rows affected)

Total execution time: 00:00:01.890

Category,BasicActivity,(No column name)
Loyalty and Helsana,Recommendation Helsana+ App,196
Challenges,Recommendation Helsana+ App,36


Conclusion: BasicActivity "Recommendation Helsana+ App" belongs to Categories "Loyalty and Helsana" as well as "Challenges". All other BasicActivity values only belong to exactly one Category.

# Analysis: Format of `ActivityDetails` depending on `BasicActivity`

In [7]:
WITH grouped AS (
    select 
        Category, 
        BasicActivity, 
        min(ActivityDetails) as Min_ActivityDetails, 
        max(ActivityDetails) as Max_ActivityDetails,
        COUNT(*) as Number_of_Records
    from UserActivity 
    group by Category, BasicActivity
)
SELECT 
    Category, 
    BasicActivity, 
    Min_ActivityDetails, 
    Max_ActivityDetails, 
    CASE
        -- Mind the typo: Unkonwn instead of Unknown
        WHEN CHARINDEX('{', Min_ActivityDetails) > 0 AND CHARINDEX('{', MAX_ActivityDetails) > 0 THEN 'JSON'
        WHEN Min_ActivityDetails = '' AND Max_ActivityDetails = '' THEN 'Empty String'
        WHEN Min_ActivityDetails = ',,' AND Max_ActivityDetails = ',,' THEN 'Empty String'
        WHEN Min_ActivityDetails = Max_ActivityDetails THEN 'Value Always: ' + IIF(Min_ActivityDetails IS NULL, '(NULL)', '"' + Min_ActivityDetails+ '"')
        ELSE 'Various Values'
    END Format_of_ActivityDetails,
    Number_of_Records
FROM grouped
ORDER BY 1,2


(61 rows affected)

Total execution time: 00:00:00.499

Category,BasicActivity,Min_ActivityDetails,Max_ActivityDetails,Format_of_ActivityDetails,Number_of_Records
Challenges,“Refer a friend” quiz,,,Empty String,624
Challenges,A balanced diet,,,Empty String,748
Challenges,Back quiz,,,Empty String,950
Challenges,Bonus for autumn challenges,,,Empty String,117
Challenges,Bonus for Coach sessions,,,Empty String,4
Challenges,Bonus for January challenges,,,Empty String,415
Challenges,Chain of survival quiz,,,Empty String,925
Challenges,"Earn points, step by step",,,Empty String,131
Challenges,Exercise quiz,,,Empty String,659
Challenges,First aid quiz,,,Empty String,470


# Focus: `UserActivity` rows with `Category` = "Movement"

In the table `UserActivity`, rows with `Category` = "Movement" have some special charcteristics. Therefore, we provide some additional information about these rows.

Below, you see all the `BasicActivity` values of rows with `Category = 'Movement'`

In [4]:
SELECT BasicActivity, COUNT(*) as Number_of_Records
FROM UserActivity
WHERE Category = 'Movement'
GROUP BY BasicActivity
ORDER BY 2 DESC  

(6 rows affected)

Total execution time: 00:00:00.119

BasicActivity,Number_of_Records
Personal exercise,132448
Daily Session Movement Coach,30483
Trails App Run,27388
My activity,6581
Movement programme Coach,1057
100'000 schritte Challenge für Spitzensportler :-),1


## `BasicActivity` values

### Personal exercise

These rows come from our version 1 system, before we had `BasicActivity` = "My activity". They represent movement activities as well but don't have any corresponding rows in the `Move` table.

### Daily Session Movement Coach

These rows come from our "Helsana Coach" app and represent sequences of exercises the user has performed. These exercises have been performed within a regular exercise plan.

### Trails App Run

These rows come from our "Helsana Trails" app and are created when a user has indicated to have run one of the "Helsana Trails".

### My activity

These rows represent movement activities recorded by trackers, and have corresponding rows in the table `Move`. 

### Movement programme Coach

These rows come from our "Helsana Coach" app and represent sequences of exercises the user has performed.

### 100'000 Schritte Challenge für Spitzensportler :-)

In the Helsana+ App, we have "challenges" which a user can get points for. Some challenges can also be "movement-related", so they will also appear under the category "Movement". (There are also different kinds of challenges which aren't "movement-related".)

# Focus: `UserActivity` rows with `Category` = "Coach Activities"

In the table `UserActivity`, rows with `Category` = "Coach Activities" have some special charcteristics. Therefore, we provide some additional information about these rows.

Below, you see all the `BasicActivity` values of rows with `Category = 'Coach Activities'`

In [17]:
WITH parsed as (
    SELECT 
        BasicActivity, 
        IIF(ISJSON(ActivityDetails) > 0, JSON_VALUE(ActivityDetails, '$.ActivityType'), ActivityDetails) as ActivityType
    FROM UserActivity
    WHERE Category = 'Coach Activities'
)
SELECT BasicActivity, ActivityType, COUNT(*) as Number_of_Records
FROM parsed
GROUP BY BasicActivity, ActivityType
ORDER BY 2 DESC  

(5 rows affected)

Total execution time: 00:00:00.247

BasicActivity,ActivityType,Number_of_Records
Daily session,Session completed,16640
Mindfulness programme,Program completed,228
Movement programme,Program completed,127
Nutrition programme,Program completed,266
Weekly Bonus,,2845
